In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
import pandas as pd
from sklearn.utils import shuffle, resample
from afqinsight.augmentation import jitter, time_warp, scaling
import tempfile

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md", "dki_mk"],
    index_col="subject_id",
    target_cols=["age", "dl_qc_score", "scan_site_id"],
    label_encode_cols=["scan_site_id"]
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1865
(1865, 7200)
(1865, 3)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-05-06 10:56:00.435244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)
y = np.array([yy[1][0] for yy in full_dataset])
qc = np.array([yy[1][1] for yy in full_dataset])
site = np.array([yy[1][2] for yy in full_dataset])

In [7]:
X = X[qc>0]
y = y[qc>0]
site = site[qc>0]

In [8]:
n_epochs = 1000

# EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    mode="min",
    patience=100
)

# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=20,
    verbose=1,
)

In [9]:
def augment_this(X, y, rounds=2): 
    new_X = X[:]
    new_y = y[:]
    for f in range(rounds): 
        aug_X = np.zeros_like(X)
        # Do each channel separately:
        for channel in range(aug_X.shape[-1]):
            this_X = X[..., channel][..., np.newaxis]
            this_X = jitter(this_X, sigma=np.mean(this_X)/25)
            this_X = scaling(this_X, sigma=np.mean(this_X)/25)
            this_X = time_warp(this_X, sigma=np.mean(this_X)/25)
            aug_X[..., channel] = this_X[...,0]
        new_X = np.concatenate([new_X, aug_X])
        new_y = np.concatenate([new_y, y])
    return new_X, new_y 

In [10]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def single_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train, X_test1, y_train, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    _, X_test2, _, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    _, X_test3, _, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [site_1]*12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

In [15]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def double_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    X_train3, X_test3, y_train3, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train1 = np.concatenate([imputer.fit_transform(X_train1[..., ii])[:, :, None] for ii in range(X_train1.shape[-1])], -1)
    X_train2 = np.concatenate([imputer.fit_transform(X_train2[..., ii])[:, :, None] for ii in range(X_train2.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    # size down evenly
    sample = y_test3.shape[0]//2
    sample1 = resample(X_train1, y_train1, n_samples=sample, replace=False)
    sample2 = resample(X_train2, y_train2, n_samples=sample, replace=False)
    X_train = np.concatenate((sample1[0], sample2[0]), axis=0)
    y_train = np.concatenate((sample1[1], sample2[1]), axis=0)
    # shuffle
    X_train, y_train = shuffle(X_train, y_train)
    X_train, y_train = augment_this(X_train, y_train)
    X_train, y_train = shuffle(X_train, y_train)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [f'{site_1}, {site_2}'] * 12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

### cnn_resnet

#### single-cross-site

In [12]:
df_resnet1, history_resnet1 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)
df_resnet2, history_resnet2 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 0, 4, X, y)
df_resnet3, history_resnet3 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 4, 0, 3, X, y)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 100, 64)      36928       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 64)      0           ['batch_normalization[0][0]']

4/4 [==============================] - ETA: 0s - loss: 81.4730 - mean_squared_error: 81.4730 - rmse: 9.0262 - mean_absolute_error: 8.1619   
Epoch 00001: val_loss improved from inf to 23169136.00000, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 5s 619ms/step - loss: 81.4730 - mean_squared_error: 81.4730 - rmse: 9.0262 - mean_absolute_error: 8.1619 - val_loss: 23169136.0000 - val_mean_squared_error: 23169136.0000 - val_rmse: 4813.4331 - val_mean_absolute_error: 4804.0498 - lr: 0.0100
Epoch 2/1000
4/4 [==============================] - ETA: 0s - loss: 13.1788 - mean_squared_error: 13.1788 - rmse: 3.6303 - mean_absolute_error: 2.8078
Epoch 00002: val_loss did not improve from 23169136.00000
4/4 [==============================] - 2s 436ms/step - loss: 13.1788 - mean_squared_error: 13.1788 - rmse: 3.6303 - mean_absolute_error: 2.8078 - val_loss: 232211952.0000 - val_mean_squared_error: 232211952.0000 - val_rmse: 15238

Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 1.4689 - mean_squared_error: 1.4689 - rmse: 1.2120 - mean_absolute_error: 0.9328
Epoch 00017: val_loss improved from 543619.00000 to 285564.34375, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 430ms/step - loss: 1.4689 - mean_squared_error: 1.4689 - rmse: 1.2120 - mean_absolute_error: 0.9328 - val_loss: 285564.3438 - val_mean_squared_error: 285564.3438 - val_rmse: 534.3822 - val_mean_absolute_error: 530.0283 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 1.3523 - mean_squared_error: 1.3523 - rmse: 1.1629 - mean_absolute_error: 0.9167
Epoch 00018: val_loss improved from 285564.34375 to 158449.54688, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 422ms/step - loss: 1.3523 - mean_squared_error: 1.3523 - rmse: 1.1629 - mean_absolute_error:

Epoch 32/1000
4/4 [==============================] - ETA: 0s - loss: 0.6042 - mean_squared_error: 0.6042 - rmse: 0.7773 - mean_absolute_error: 0.6172
Epoch 00032: val_loss did not improve from 389.09836
4/4 [==============================] - 2s 474ms/step - loss: 0.6042 - mean_squared_error: 0.6042 - rmse: 0.7773 - mean_absolute_error: 0.6172 - val_loss: 415.8896 - val_mean_squared_error: 415.8896 - val_rmse: 20.3934 - val_mean_absolute_error: 19.2816 - lr: 0.0100
Epoch 33/1000
4/4 [==============================] - ETA: 0s - loss: 0.6867 - mean_squared_error: 0.6867 - rmse: 0.8287 - mean_absolute_error: 0.6483
Epoch 00033: val_loss improved from 389.09836 to 92.84183, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 458ms/step - loss: 0.6867 - mean_squared_error: 0.6867 - rmse: 0.8287 - mean_absolute_error: 0.6483 - val_loss: 92.8418 - val_mean_squared_error: 92.8418 - val_rmse: 9.6354 - val_mean_absolute_error: 

Epoch 49/1000
4/4 [==============================] - ETA: 0s - loss: 0.2948 - mean_squared_error: 0.2948 - rmse: 0.5430 - mean_absolute_error: 0.4275
Epoch 00049: val_loss did not improve from 8.86772
4/4 [==============================] - 2s 427ms/step - loss: 0.2948 - mean_squared_error: 0.2948 - rmse: 0.5430 - mean_absolute_error: 0.4275 - val_loss: 13.0964 - val_mean_squared_error: 13.0964 - val_rmse: 3.6189 - val_mean_absolute_error: 2.7594 - lr: 0.0100
Epoch 50/1000
4/4 [==============================] - ETA: 0s - loss: 0.3993 - mean_squared_error: 0.3993 - rmse: 0.6319 - mean_absolute_error: 0.4878
Epoch 00050: val_loss improved from 8.86772 to 8.84559, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 429ms/step - loss: 0.3993 - mean_squared_error: 0.3993 - rmse: 0.6319 - mean_absolute_error: 0.4878 - val_loss: 8.8456 - val_mean_squared_error: 8.8456 - val_rmse: 2.9742 - val_mean_absolute_error: 2.2086 - lr

...

Epoch 450/1000
2/2 [==============================] - ETA: 0s - loss: 0.0184 - mean_squared_error: 0.0184 - rmse: 0.1356 - mean_absolute_error: 0.1011
Epoch 00450: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 155ms/step - loss: 0.0184 - mean_squared_error: 0.0184 - rmse: 0.1356 - mean_absolute_error: 0.1011 - val_loss: 5.0865 - val_mean_squared_error: 5.0865 - val_rmse: 2.2553 - val_mean_absolute_error: 1.8473 - lr: 4.8828e-06
Epoch 451/1000
2/2 [==============================] - ETA: 0s - loss: 0.2354 - mean_squared_error: 0.2354 - rmse: 0.4852 - mean_absolute_error: 0.3548
Epoch 00451: val_loss did not improve from 5.00231
2/2 [==============================] - 1s 160ms/step - loss: 0.2354 - mean_squared_error: 0.2354 - rmse: 0.4852 - mean_absolute_error: 0.3548 - val_loss: 5.0877 - val_mean_squared_error: 5.0877 - val_rmse: 2.2556 - val_mean_absolute_error: 1.8482 - lr: 4.8828e-06
Epoch 452/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 468/1000
2/2 [==============================] - ETA: 0s - loss: 0.0288 - mean_squared_error: 0.0288 - rmse: 0.1696 - mean_absolute_error: 0.1245
Epoch 00468: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 145ms/step - loss: 0.0288 - mean_squared_error: 0.0288 - rmse: 0.1696 - mean_absolute_error: 0.1245 - val_loss: 5.0656 - val_mean_squared_error: 5.0656 - val_rmse: 2.2507 - val_mean_absolute_error: 1.8369 - lr: 2.4414e-06
Epoch 469/1000
2/2 [==============================] - ETA: 0s - loss: 0.1187 - mean_squared_error: 0.1187 - rmse: 0.3445 - mean_absolute_error: 0.2475
Epoch 00469: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 157ms/step - loss: 0.1187 - mean_squared_error: 0.1187 - rmse: 0.3445 - mean_absolute_error: 0.2475 - val_loss: 5.0699 - val_mean_squared_error: 5.0699 - val_rmse: 2.2516 - val_mean_absolute_error: 1.8388 - lr: 2.4414e-06
Epoch 470/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 486/1000
2/2 [==============================] - ETA: 0s - loss: 0.2470 - mean_squared_error: 0.2470 - rmse: 0.4970 - mean_absolute_error: 0.3763
Epoch 00486: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 159ms/step - loss: 0.2470 - mean_squared_error: 0.2470 - rmse: 0.4970 - mean_absolute_error: 0.3763 - val_loss: 5.0499 - val_mean_squared_error: 5.0499 - val_rmse: 2.2472 - val_mean_absolute_error: 1.8326 - lr: 1.2207e-06
Epoch 487/1000
2/2 [==============================] - ETA: 0s - loss: 0.0215 - mean_squared_error: 0.0215 - rmse: 0.1466 - mean_absolute_error: 0.1065
Epoch 00487: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 155ms/step - loss: 0.0215 - mean_squared_error: 0.0215 - rmse: 0.1466 - mean_absolute_error: 0.1065 - val_loss: 5.0539 - val_mean_squared_error: 5.0539 - val_rmse: 2.2481 - val_mean_absolute_error: 1.8334 - lr: 1.2207e-06
Epoch 488/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 504/1000
2/2 [==============================] - ETA: 0s - loss: 0.4081 - mean_squared_error: 0.4081 - rmse: 0.6388 - mean_absolute_error: 0.4703
Epoch 00504: val_loss did not improve from 5.00231

Epoch 00504: ReduceLROnPlateau reducing learning rate to 6.103515488575795e-07.
2/2 [==============================] - 0s 148ms/step - loss: 0.4081 - mean_squared_error: 0.4081 - rmse: 0.6388 - mean_absolute_error: 0.4703 - val_loss: 5.0728 - val_mean_squared_error: 5.0728 - val_rmse: 2.2523 - val_mean_absolute_error: 1.8373 - lr: 1.2207e-06
Epoch 505/1000
2/2 [==============================] - ETA: 0s - loss: 0.0224 - mean_squared_error: 0.0224 - rmse: 0.1497 - mean_absolute_error: 0.1063
Epoch 00505: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 146ms/step - loss: 0.0224 - mean_squared_error: 0.0224 - rmse: 0.1497 - mean_absolute_error: 0.1063 - val_loss: 5.0742 - val_mean_squared_error: 5.0742 - val_rmse: 2.2526 - val_mean_absolute_error: 1.8377 - lr

Epoch 522/1000
2/2 [==============================] - ETA: 0s - loss: 0.0392 - mean_squared_error: 0.0392 - rmse: 0.1981 - mean_absolute_error: 0.1419
Epoch 00522: val_loss did not improve from 5.00231
2/2 [==============================] - 1s 175ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - rmse: 0.1981 - mean_absolute_error: 0.1419 - val_loss: 5.0386 - val_mean_squared_error: 5.0386 - val_rmse: 2.2447 - val_mean_absolute_error: 1.8230 - lr: 6.1035e-07
Epoch 523/1000
2/2 [==============================] - ETA: 0s - loss: 0.0231 - mean_squared_error: 0.0231 - rmse: 0.1521 - mean_absolute_error: 0.1048
Epoch 00523: val_loss did not improve from 5.00231
2/2 [==============================] - 0s 154ms/step - loss: 0.0231 - mean_squared_error: 0.0231 - rmse: 0.1521 - mean_absolute_error: 0.1048 - val_loss: 5.0375 - val_mean_squared_error: 5.0375 - val_rmse: 2.2444 - val_mean_absolute_error: 1.8225 - lr: 6.1035e-07
Epoch 524/1000
2/2 [==============================] - ETA: 0s - loss

In [13]:
df_resnet_1 = (df_resnet1.merge(df_resnet2, how='outer')).merge(df_resnet3, how='outer')

#### double-cross-site

In [16]:
df_resnet4, hisory_resnet4 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 4, 0, X, y)
df_resnet5, hisory_resnet5 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 4, 3, X, y)
df_resnet6, hisory_resnet6 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d_33 (Conv1D)             (None, 100, 64)      36928       ['input_4[0][0]']                
                                                                                                  
 batch_normalization_36 (BatchN  (None, 100, 64)     256         ['conv1d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_36 (Activation)     (None, 100, 64)      0           ['batch_normalization_36[0]

3/3 [==============================] - ETA: 0s - loss: 103.0358 - mean_squared_error: 103.0358 - rmse: 10.1507 - mean_absolute_error: 9.3497 
Epoch 00001: val_loss improved from inf to 1046869.31250, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpjaexw9zt.h5
3/3 [==============================] - 4s 514ms/step - loss: 103.0358 - mean_squared_error: 103.0358 - rmse: 10.1507 - mean_absolute_error: 9.3497 - val_loss: 1046869.3125 - val_mean_squared_error: 1046869.3125 - val_rmse: 1023.1663 - val_mean_absolute_error: 1012.9980 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - ETA: 0s - loss: 26.5037 - mean_squared_error: 26.5037 - rmse: 5.1482 - mean_absolute_error: 4.3272
Epoch 00002: val_loss did not improve from 1046869.31250
3/3 [==============================] - 1s 323ms/step - loss: 26.5037 - mean_squared_error: 26.5037 - rmse: 5.1482 - mean_absolute_error: 4.3272 - val_loss: 193622928.0000 - val_mean_squared_error: 193622928.0000 - val_rmse: 13914

Epoch 17/1000
3/3 [==============================] - ETA: 0s - loss: 1.2505 - mean_squared_error: 1.2505 - rmse: 1.1183 - mean_absolute_error: 0.8902
Epoch 00017: val_loss improved from 178445.84375 to 96222.64062, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpjaexw9zt.h5
3/3 [==============================] - 1s 356ms/step - loss: 1.2505 - mean_squared_error: 1.2505 - rmse: 1.1183 - mean_absolute_error: 0.8902 - val_loss: 96222.6406 - val_mean_squared_error: 96222.6406 - val_rmse: 310.1978 - val_mean_absolute_error: 301.4501 - lr: 0.0100
Epoch 18/1000
3/3 [==============================] - ETA: 0s - loss: 1.0627 - mean_squared_error: 1.0627 - rmse: 1.0309 - mean_absolute_error: 0.8250
Epoch 00018: val_loss improved from 96222.64062 to 43750.51172, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpjaexw9zt.h5
3/3 [==============================] - 1s 359ms/step - loss: 1.0627 - mean_squared_error: 1.0627 - rmse: 1.0309 - mean_absolute_error: 0.82

Epoch 32/1000
3/3 [==============================] - ETA: 0s - loss: 0.3334 - mean_squared_error: 0.3334 - rmse: 0.5774 - mean_absolute_error: 0.4569
Epoch 00032: val_loss did not improve from 16.96515
3/3 [==============================] - 1s 341ms/step - loss: 0.3334 - mean_squared_error: 0.3334 - rmse: 0.5774 - mean_absolute_error: 0.4569 - val_loss: 21.9640 - val_mean_squared_error: 21.9640 - val_rmse: 4.6866 - val_mean_absolute_error: 3.7480 - lr: 0.0100
Epoch 33/1000
3/3 [==============================] - ETA: 0s - loss: 0.3387 - mean_squared_error: 0.3387 - rmse: 0.5820 - mean_absolute_error: 0.4550
Epoch 00033: val_loss did not improve from 16.96515
3/3 [==============================] - 1s 347ms/step - loss: 0.3387 - mean_squared_error: 0.3387 - rmse: 0.5820 - mean_absolute_error: 0.4550 - val_loss: 32.9119 - val_mean_squared_error: 32.9119 - val_rmse: 5.7369 - val_mean_absolute_error: 4.7124 - lr: 0.0100
Epoch 34/1000
3/3 [==============================] - ETA: 0s - loss: 0.3

Epoch 50/1000
3/3 [==============================] - ETA: 0s - loss: 0.2402 - mean_squared_error: 0.2402 - rmse: 0.4901 - mean_absolute_error: 0.3872
Epoch 00050: val_loss did not improve from 11.09401
3/3 [==============================] - 1s 343ms/step - loss: 0.2402 - mean_squared_error: 0.2402 - rmse: 0.4901 - mean_absolute_error: 0.3872 - val_loss: 11.1804 - val_mean_squared_error: 11.1804 - val_rmse: 3.3437 - val_mean_absolute_error: 2.6127 - lr: 0.0100
Epoch 51/1000
3/3 [==============================] - ETA: 0s - loss: 0.2150 - mean_squared_error: 0.2150 - rmse: 0.4637 - mean_absolute_error: 0.3749
Epoch 00051: val_loss did not improve from 11.09401
3/3 [==============================] - 1s 337ms/step - loss: 0.2150 - mean_squared_error: 0.2150 - rmse: 0.4637 - mean_absolute_error: 0.3749 - val_loss: 12.9055 - val_mean_squared_error: 12.9055 - val_rmse: 3.5924 - val_mean_absolute_error: 2.9330 - lr: 0.0100
Epoch 52/1000
3/3 [==============================] - ETA: 0s - loss: 0.5

...

Epoch 840/1000
1/1 [==============================] - ETA: 0s - loss: 2.0597e-04 - mean_squared_error: 2.0597e-04 - rmse: 0.0144 - mean_absolute_error: 0.0054
Epoch 00840: val_loss did not improve from 0.59552
1/1 [==============================] - 0s 332ms/step - loss: 2.0597e-04 - mean_squared_error: 2.0597e-04 - rmse: 0.0144 - mean_absolute_error: 0.0054 - val_loss: 0.5956 - val_mean_squared_error: 0.5956 - val_rmse: 0.7717 - val_mean_absolute_error: 0.6119 - lr: 0.0012
Epoch 841/1000
1/1 [==============================] - ETA: 0s - loss: 2.0508e-04 - mean_squared_error: 2.0508e-04 - rmse: 0.0143 - mean_absolute_error: 0.0054
Epoch 00841: val_loss did not improve from 0.59552
1/1 [==============================] - 0s 329ms/step - loss: 2.0508e-04 - mean_squared_error: 2.0508e-04 - rmse: 0.0143 - mean_absolute_error: 0.0054 - val_loss: 0.5956 - val_mean_squared_error: 0.5956 - val_rmse: 0.7717 - val_mean_absolute_error: 0.6119 - lr: 0.0012
Epoch 842/1000
1/1 [========================

Epoch 855/1000
1/1 [==============================] - ETA: 0s - loss: 1.9813e-04 - mean_squared_error: 1.9813e-04 - rmse: 0.0141 - mean_absolute_error: 0.0053
Epoch 00855: val_loss improved from 0.59516 to 0.59514, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpe2teu_4j.h5
1/1 [==============================] - 0s 366ms/step - loss: 1.9813e-04 - mean_squared_error: 1.9813e-04 - rmse: 0.0141 - mean_absolute_error: 0.0053 - val_loss: 0.5951 - val_mean_squared_error: 0.5951 - val_rmse: 0.7715 - val_mean_absolute_error: 0.6122 - lr: 6.2500e-04
Epoch 856/1000
1/1 [==============================] - ETA: 0s - loss: 1.9770e-04 - mean_squared_error: 1.9770e-04 - rmse: 0.0141 - mean_absolute_error: 0.0053
Epoch 00856: val_loss improved from 0.59514 to 0.59514, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpe2teu_4j.h5
1/1 [==============================] - 0s 365ms/step - loss: 1.9770e-04 - mean_squared_error: 1.9770e-04 - rmse: 0.0141 - mean_absolute_er

Epoch 870/1000
1/1 [==============================] - ETA: 0s - loss: 1.9186e-04 - mean_squared_error: 1.9186e-04 - rmse: 0.0139 - mean_absolute_error: 0.0051
Epoch 00870: val_loss did not improve from 0.59490
1/1 [==============================] - 0s 356ms/step - loss: 1.9186e-04 - mean_squared_error: 1.9186e-04 - rmse: 0.0139 - mean_absolute_error: 0.0051 - val_loss: 0.5949 - val_mean_squared_error: 0.5949 - val_rmse: 0.7713 - val_mean_absolute_error: 0.6125 - lr: 6.2500e-04
Epoch 871/1000
1/1 [==============================] - ETA: 0s - loss: 1.9145e-04 - mean_squared_error: 1.9145e-04 - rmse: 0.0138 - mean_absolute_error: 0.0051
Epoch 00871: val_loss improved from 0.59490 to 0.59490, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpe2teu_4j.h5
1/1 [==============================] - 0s 359ms/step - loss: 1.9145e-04 - mean_squared_error: 1.9145e-04 - rmse: 0.0138 - mean_absolute_error: 0.0051 - val_loss: 0.5949 - val_mean_squared_error: 0.5949 - val_rmse: 0.7713 - 

1/1 [==============================] - ETA: 0s - loss: 1.8580e-04 - mean_squared_error: 1.8580e-04 - rmse: 0.0136 - mean_absolute_error: 0.0050
Epoch 00886: val_loss did not improve from 0.59488
1/1 [==============================] - 0s 342ms/step - loss: 1.8580e-04 - mean_squared_error: 1.8580e-04 - rmse: 0.0136 - mean_absolute_error: 0.0050 - val_loss: 0.5949 - val_mean_squared_error: 0.5949 - val_rmse: 0.7713 - val_mean_absolute_error: 0.6128 - lr: 3.1250e-04
Epoch 887/1000
1/1 [==============================] - ETA: 0s - loss: 1.8560e-04 - mean_squared_error: 1.8560e-04 - rmse: 0.0136 - mean_absolute_error: 0.0050
Epoch 00887: val_loss did not improve from 0.59488
1/1 [==============================] - 0s 329ms/step - loss: 1.8560e-04 - mean_squared_error: 1.8560e-04 - rmse: 0.0136 - mean_absolute_error: 0.0050 - val_loss: 0.5949 - val_mean_squared_error: 0.5949 - val_rmse: 0.7713 - val_mean_absolute_error: 0.6128 - lr: 3.1250e-04
Epoch 888/1000
1/1 [==============================]

1/1 [==============================] - ETA: 0s - loss: 1.8244e-04 - mean_squared_error: 1.8244e-04 - rmse: 0.0135 - mean_absolute_error: 0.0050
Epoch 00903: val_loss did not improve from 0.59488

Epoch 00903: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
1/1 [==============================] - 0s 352ms/step - loss: 1.8244e-04 - mean_squared_error: 1.8244e-04 - rmse: 0.0135 - mean_absolute_error: 0.0050 - val_loss: 0.5951 - val_mean_squared_error: 0.5951 - val_rmse: 0.7714 - val_mean_absolute_error: 0.6131 - lr: 3.1250e-04
Epoch 904/1000
1/1 [==============================] - ETA: 0s - loss: 1.8225e-04 - mean_squared_error: 1.8225e-04 - rmse: 0.0135 - mean_absolute_error: 0.0050
Epoch 00904: val_loss did not improve from 0.59488
1/1 [==============================] - 0s 342ms/step - loss: 1.8225e-04 - mean_squared_error: 1.8225e-04 - rmse: 0.0135 - mean_absolute_error: 0.0050 - val_loss: 0.5951 - val_mean_squared_error: 0.5951 - val_rmse: 0.7714 - val_mean_absolute_

In [17]:
df_resnet_2 = (df_resnet4.merge(df_resnet5, how='outer')).merge(df_resnet6, how='outer')

#### cost0

In [18]:
resnet_cost0 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 0)]

In [19]:
resnet_cost0

,Model,Train_site,Test_site,Metric,Value
2,cnn_resnet,0,0,MAE,1.604714
18,cnn_resnet,3,0,MAE,4.560169
30,cnn_resnet,4,0,MAE,2.825388


In [20]:
resnet_cost0_3 = (resnet_cost0[resnet_cost0['Train_site'] == 3]['Value'].values - 
                  resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)
resnet_cost0_4 = (resnet_cost0[resnet_cost0['Train_site'] == 4]['Value'].values - 
                  resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)

In [21]:
resnet_cost0_3, resnet_cost0_4

(array([2.95545554]), array([1.22067404]))

In [22]:
resnet_cost0_mean = np.mean([resnet_cost0_3, resnet_cost0_4])

In [23]:
resnet_cost0_0 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 0)]

In [24]:
resnet_cost0_0

,Model,Train_site,Test_site,Metric,Value
10,cnn_resnet,"3, 4",0,MAE,2.710760
14,cnn_resnet,"0, 4",0,MAE,2.247498
26,cnn_resnet,"0, 3",0,MAE,2.284334


In [25]:
resnet_cost0_34 = (resnet_cost0_0[resnet_cost0_0['Train_site'] == '3, 4']['Value'].values - 
                   resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)

In [51]:
resnet_cost0_mean, resnet_cost0_34

(2.0880647897720337, array([1.10604668]))

#### cost3

In [27]:
resnet_cost3 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 3)]

In [28]:
resnet_cost3_0 = (resnet_cost3[resnet_cost3['Train_site'] == 0]['Value'].values - 
                  resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)
resnet_cost3_4 = (resnet_cost3[resnet_cost3['Train_site'] == 4]['Value'].values - 
                  resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)

In [29]:
resnet_cost3_0, resnet_cost3_4

(array([1.24640751]), array([1.59477758]))

In [30]:
resnet_cost3_mean = np.mean([resnet_cost3_0, resnet_cost3_4])

In [31]:
resnet_cost3_3 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 3)]

In [32]:
resnet_cost3_3

,Model,Train_site,Test_site,Metric,Value
2,cnn_resnet,"3, 4",3,MAE,2.180389
22,cnn_resnet,"0, 4",3,MAE,3.117193
30,cnn_resnet,"0, 3",3,MAE,2.271742


In [33]:
resnet_cost3_04 = (resnet_cost3_3[resnet_cost3_3['Train_site'] == '0, 4']['Value'].values - 
                   resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)

In [52]:
resnet_cost3_mean, resnet_cost3_04

(1.4205925464630127, array([1.13324285]))

#### cost4

In [41]:
resnet_cost4 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 4)]

In [47]:
resnet_cost4

,Model,Train_site,Test_site,Metric,Value
10,cnn_resnet,0,4,MAE,2.371763
22,cnn_resnet,3,4,MAE,2.126879
26,cnn_resnet,4,4,MAE,2.122975


In [42]:
resnet_cost4_0 = (resnet_cost4[resnet_cost4['Train_site'] == 0]['Value'].values - 
                  resnet_cost4[resnet_cost4['Train_site'] == 4]['Value'].values)
resnet_cost4_3 = (resnet_cost4[resnet_cost4['Train_site'] == 3]['Value'].values - 
                  resnet_cost4[resnet_cost4['Train_site'] == 4]['Value'].values)

In [43]:
resnet_cost4_mean = np.mean([resnet_cost4_0, resnet_cost4_3])

In [44]:
resnet_cost4_4 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 4)]

In [49]:
resnet_cost4_4

,Model,Train_site,Test_site,Metric,Value
6,cnn_resnet,"3, 4",4,MAE,2.163166
18,cnn_resnet,"0, 4",4,MAE,2.240731
34,cnn_resnet,"0, 3",4,MAE,3.038536


In [45]:
resnet_cost4_03 = (resnet_cost4_4[resnet_cost4_4['Train_site'] == '0, 3']['Value'].values - 
                   resnet_cost4[resnet_cost4['Train_site'] == 4]['Value'].values)

In [50]:
resnet_cost4_mean, resnet_cost4_03

(0.12634611129760742, array([0.91556096]))